# Imports

In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np

from pfc_packages.pointNet import *
from pfc_packages.utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Hyperparameters

In [2]:
NUM_CLASSES = 20
EPOCHS = 1000
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model configs

In [3]:
# ==== LOAD DATA ====
unprocessed_train_dataset = Dataset("../../datasets/semantic-kitti", load_cluster=True)

# ==== MODEL ====
model = PointNetSeg(num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=LR)

# Pre-process clusters

In [12]:
processed_train_dataset = []
count = 0
for frame in tqdm(unprocessed_train_dataset, desc="Processando frames"):
    processed_train_dataset.append(
        extract_features_from_clusters(frame, min_cluster_size=3)
    )
    count += 1
    if count == 5:
        break

Processando frames:   1%|▏         | 4/271 [00:05<05:53,  1.32s/it]


# Train model

In [13]:
model.train()  # ainda assim necessário
for m in model.modules():
    if isinstance(m, torch.nn.Dropout):
        m.p = 0.0  # ou apenas comente o dropout na definição do modelo

In [14]:
for epoch in tqdm(range(EPOCHS), desc="Epochs", unit="epoch"):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_points = 0

    for item in tqdm(processed_train_dataset, desc=f"Frame (Epoch {epoch+1}/{EPOCHS})", unit="frame", leave=False):
    # for item in tqdm(processed_train_dataset, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="item"):
        data = torch.tensor(item[:, :3], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        label = torch.tensor(item[:, 3], dtype=torch.long).unsqueeze(0).to(DEVICE)

        optimizer.zero_grad()
        pred = model(data)
        pred = pred.view(-1, NUM_CLASSES)
        label = label.view(-1)

        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (pred.argmax(dim=1) == label).sum().item()
        total_points += label.numel()

        acc = total_correct / total_points

        if epoch % 100 == 0:
            print(f"[Epoch {epoch+1:05d}] Loss: {loss.item():.4f} | Acc: {acc*100:.2f}%")

print(f"[Epoch {epoch+1:05d}] Loss: {loss.item():.4f} | Acc: {acc*100:.2f}%")
# Opcional: salvar checkpoints
# torch.save(model.state_dict(), f"checkpoints/pointnet_epoch_{epoch+1:03d}.pth")

Epochs:   0%|          | 1/1000 [00:00<01:53,  8.77epoch/s]

[Epoch 00001] Loss: 0.0021 | Acc: 100.00%
[Epoch 00001] Loss: 0.2288 | Acc: 97.44%
[Epoch 00001] Loss: 0.9275 | Acc: 91.27%
[Epoch 00001] Loss: 1.5928 | Acc: 86.05%
[Epoch 00001] Loss: 0.9895 | Acc: 84.13%


Epochs:  10%|█         | 101/1000 [00:02<00:22, 39.27epoch/s]

[Epoch 00101] Loss: 0.0684 | Acc: 98.06%
[Epoch 00101] Loss: 0.0870 | Acc: 97.36%
[Epoch 00101] Loss: 0.0724 | Acc: 97.69%
[Epoch 00101] Loss: 0.0769 | Acc: 97.64%
[Epoch 00101] Loss: 0.1164 | Acc: 97.28%


Epochs:  20%|██        | 201/1000 [00:05<00:20, 38.96epoch/s]

...

[Epoch 00801] Loss: 0.0307 | Acc: 98.87%
[Epoch 00801] Loss: 0.0175 | Acc: 99.28%
[Epoch 00801] Loss: 0.0177 | Acc: 99.42%
[Epoch 00801] Loss: 0.0279 | Acc: 99.29%
[Epoch 00801] Loss: 0.0280 | Acc: 99.28%


Epochs:  90%|█████████ | 900/1000 [00:24<00:02, 36.67epoch/s]

[Epoch 00901] Loss: 0.0113 | Acc: 99.84%
[Epoch 00901] Loss: 0.0102 | Acc: 99.76%
[Epoch 00901] Loss: 0.0087 | Acc: 99.84%
[Epoch 00901] Loss: 0.0161 | Acc: 99.69%
[Epoch 00901] Loss: 0.0138 | Acc: 99.63%


Epochs: 100%|██████████| 1000/1000 [00:27<00:00, 36.79epoch/s]

[Epoch 01000] Loss: 0.0455 | Acc: 98.88%


# Inference

In [15]:
point_cloud = unprocessed_train_dataset[0]
cluster_pc = extract_features_from_clusters(point_cloud, drop_incosistentes=True)

data = torch.tensor(cluster_pc[:, :3], dtype=torch.float32).unsqueeze(0).to(DEVICE)
label = torch.tensor(cluster_pc[:, 3], dtype=torch.long).unsqueeze(0).to(DEVICE)

# # Carregar o modelo treinado
# model = PointNetSeg(num_classes=NUM_CLASSES).to(DEVICE)
# model.load_state_dict(torch.load("checkpoints/pointnet_epoch_001.pth"))

model.eval()
with torch.no_grad():
    pred = model(data)
    pred = pred.view(-1, NUM_CLASSES)

    # pred_labels = torch.argmax(pred, dim=1).cpu().numpy()
    pred = pred.argmax(dim=1)

# Agora, substituindo os rótulos originais pelos rótulos preditos
# Mantendo as coordenadas xyz no formato original e substituindo o label
result = np.hstack((data.squeeze(0).cpu().numpy(), pred.cpu().numpy().reshape(-1, 1)))

In [16]:
result

array([[42.63254929, 12.12786674,  1.71147394, 15.        ],
       [60.38326645, 17.59372711,  2.34032941,  9.        ],
       [45.88910675, 18.42118454,  1.88575268, 15.        ],
       ...,
       [ 2.55438852, 12.3309288 , -1.74443364, 17.        ],
       [ 2.43118119, 12.56003189, -1.77464926, 17.        ],
       [ 2.27099586, 12.35544777, -1.73908126, 17.        ]],
      shape=(1687, 4))

In [17]:
run_viz(
    result,
    cluster_viz=True,
    show_true_label=True,
    point_size=3,
    # show_min_max=True,
)

In [10]:
run_viz(
    cluster_pc,
    cluster_viz=True,
    show_true_label=True,
    point_size=3,
    # show_min_max=True,
)